In [3]:
import numpy as np
print(np.__version__)


2.2.3


In [1]:
import soundfile as sf

file_path = "C:\\Users\\jadit\\Downloads\\3 (11).wav"

# Read audio file using soundfile
data, samplerate = sf.read(file_path)
channels = data.shape[1] if data.ndim > 1 else 1

print(f"Number of channels: {channels}")


Number of channels: 2


In [2]:
import os
import numpy as np
import pandas as pd
import librosa
from joblib import Parallel, delayed
import soundfile as sf
from tqdm import tqdm

# Path to the root directory containing subdirectories with WAV files
ROOT_DIR = 'test'
OUTPUT_CSV = 'extracted_features.csv'
SPEED_OF_SOUND = 343  # Speed of sound in m/s

def compute_tdoa(y, sr):
    if y.ndim < 2:
        return None
    
    # Split into left and right channels
    channel_1 = y[:, 0]
    channel_2 = y[:, 1]
    
    correlation = np.correlate(channel_1, channel_2, mode="full")
    delay = np.argmax(correlation) - (len(channel_1) - 1)
    
    tdoa = delay / sr
    return tdoa

def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None, mono=False)
        if y.ndim == 1:  # Convert mono to stereo for TDOA
            y = np.stack([y, y], axis=0)
            
        if np.all(y == 0):
            print(f"Skipping silent file: {file_path}")
            return None

        # TDOA
        tdoa = compute_tdoa(y, sr)

        # Zero Crossing Rate
        zcr = np.mean(librosa.feature.zero_crossing_rate(y[0]))

        # Spectral Features
        spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y[0], sr=sr))
        spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y[0], sr=sr))
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y[0], sr=sr))
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y[0], sr=sr))

        # Entropy of Energy
        ste = np.sum(y[0]**2) / len(y[0])
        entropy_energy = -np.sum((y[0]**2 / ste) * np.log2(y[0]**2 / ste + 1e-10))

        # MFCCs (First 5)
        mfccs = librosa.feature.mfcc(y=y[0], sr=sr, n_mfcc=5).mean(axis=1)

        # Result in dictionary form
        result = {
            "Filename": os.path.basename(file_path),
            "TDOA": tdoa,
            "Zero_Crossing_Rate": zcr,
            "Spectral_Centroid": spectral_centroid,
            "Spectral_Bandwidth": spectral_bandwidth,
            "Spectral_Contrast": spectral_contrast,
            "Spectral_Rolloff": spectral_rolloff,
            "Entropy_Energy": entropy_energy,
            "Short_Time_Energy": ste,
            "MFCC_1": mfccs[0],
            "MFCC_2": mfccs[1],
            "MFCC_3": mfccs[2],
            "MFCC_4": mfccs[3],
            "MFCC_5": mfccs[4]
        }
        return result

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def get_wav_files(root_dir):
    wav_files = []
    for root, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.wav'):
                wav_files.append(os.path.join(root, file))
    return wav_files

def main():
    wav_files = get_wav_files(ROOT_DIR)
    print(f"Found {len(wav_files)} WAV files.")

    results = []
    # Use tqdm to track progress
    for file in tqdm(wav_files, desc="Processing files"):
        result = extract_features(file)
        if result is not None:
            results.append(result)

    # Convert to DataFrame and Save
    df = pd.DataFrame(results)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"\n✅ Saved extracted features to {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


Found 851 WAV files.


Processing files: 100%|██████████| 851/851 [01:28<00:00,  9.61it/s]


✅ Saved extracted features to extracted_features.csv


In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from joblib import Parallel, delayed
import soundfile as sf
from tqdm import tqdm

# Path to the root directory containing subdirectories with WAV files
ROOT_DIR = 'test'
OUTPUT_CSV = 'extracted_features_with_TDOA.csv'
SPEED_OF_SOUND = 343  # Speed of sound in m/s

def add_time_delay(y, sr, delay_sec=0.001):  # Default delay = 1ms
    """Add a time delay to the second channel to simulate DOA conditions."""
    delay_samples = int(delay_sec * sr)

    # Pad the delayed signal to maintain array length
    delayed_channel = np.pad(y[1], (delay_samples, 0))[:len(y[1])]
    
    return np.vstack([y[0], delayed_channel])

def compute_tdoa(y, sr):
    if y.ndim < 2:
        return None
    
    # Split into left and right channels
    channel_1 = y[0]
    channel_2 = y[1]
    
    correlation = np.correlate(channel_1, channel_2, mode="full")
    delay = np.argmax(correlation) - (len(channel_1) - 1)
    
    tdoa = delay / sr
    return tdoa

def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None, mono=False)
        if y.ndim == 1:  # Convert mono to stereo for TDOA
            y = np.stack([y, y], axis=0)
            
        if np.all(y == 0):
            print(f"Skipping silent file: {file_path}")
            return None

        # Add Time Delay for DOA Simulation
        y = add_time_delay(y, sr, delay_sec=0.001)  # Delay of 1ms (adjustable)

        # TDOA
        tdoa = compute_tdoa(y, sr)

        # Zero Crossing Rate
        zcr = np.mean(librosa.feature.zero_crossing_rate(y[0]))

        # Spectral Features
        spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y[0], sr=sr))
        spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y[0], sr=sr))
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y[0], sr=sr))
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y[0], sr=sr))

        # Entropy of Energy
        ste = np.sum(y[0]**2) / len(y[0])
        entropy_energy = -np.sum((y[0]**2 / ste) * np.log2(y[0]**2 / ste + 1e-10))

        # MFCCs (First 5)
        mfccs = librosa.feature.mfcc(y=y[0], sr=sr, n_mfcc=5).mean(axis=1)

        # Result in dictionary form
        result = {
            "Filename": os.path.basename(file_path),
            "TDOA": tdoa,
            "Zero_Crossing_Rate": zcr,
            "Spectral_Centroid": spectral_centroid,
            "Spectral_Bandwidth": spectral_bandwidth,
            "Spectral_Contrast": spectral_contrast,
            "Spectral_Rolloff": spectral_rolloff,
            "Entropy_Energy": entropy_energy,
            "Short_Time_Energy": ste,
            "MFCC_1": mfccs[0],
            "MFCC_2": mfccs[1],
            "MFCC_3": mfccs[2],
            "MFCC_4": mfccs[3],
            "MFCC_5": mfccs[4]
        }
        return result

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def get_wav_files(root_dir):
    wav_files = []
    for root, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.wav'):
                wav_files.append(os.path.join(root, file))
    return wav_files

def main():
    wav_files = get_wav_files(ROOT_DIR)
    print(f"Found {len(wav_files)} WAV files.")

    results = []
    # Use tqdm to track progress
    for file in tqdm(wav_files, desc="Processing files"):
        result = extract_features(file)
        if result is not None:
            results.append(result)

    # Convert to DataFrame and Save
    df = pd.DataFrame(results)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"\n✅ Saved extracted features to {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


Found 851 WAV files.


Processing files:   6%|▌         | 52/851 [01:06<15:34,  1.17s/it] 

In [5]:
import pandas as pd

# Load the XLSX file
input_file = 'BGG1_features_output.xlsx'
output_file = 'BGG_features_output.csv'

# Read Excel file
df = pd.read_excel(input_file)

# Save as CSV
df.to_csv(output_file, index=False)

print(f"Converted '{input_file}' to '{output_file}'")


Converted 'BGG1_features_output.xlsx' to 'BGG_features_output.csv'


In [11]:
import os
import numpy as np
import pandas as pd
import librosa
import gc
from tqdm import tqdm
import soundfile as sf

# Reduce numba thread conflicts
os.environ["NUMBA_NUM_THREADS"] = "1"

# Bias range for TDOA_1_2
BIAS_1_2 = (-2e-5, 2e-5)

def compute_tdoa(audio, sr):
    if audio.shape[0] < 2:
        return None
    
    channel_1 = audio[0, :]
    channel_2 = audio[1, :]
    
    correlation = np.correlate(channel_1, channel_2, mode="full")
    delay = np.argmax(correlation) - (len(channel_1) - 1)
    tdoa = delay / sr
    return tdoa

def extract_features(file_path):
    try:
        # Load using soundfile to avoid memory overflow
        with sf.SoundFile(file_path) as f:
            y = f.read(always_2d=True)
            sr = f.samplerate

        if y.shape[0] == 1:
            y = np.array([y[0], y[0]])  # Duplicate channel for mono files
        
        # TDOA calculation (only 1_2)
        tdoa_1_2 = compute_tdoa(y, sr) or 0
        tdoa_1_2 += np.random.uniform(*BIAS_1_2)  # ✅ Add small bias
        
        # Zero Crossing Rate
        zcr = np.mean(librosa.feature.zero_crossing_rate(y[0]))
        
        # Spectral Features
        spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y[0], sr=sr))
        spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y[0], sr=sr))
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y[0], sr=sr))
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y[0], sr=sr))

        # Entropy of Energy
        energy = np.square(y[0])
        prob_energy = energy / np.sum(energy)
        entropy_energy = -np.sum(prob_energy * np.log2(prob_energy + 1e-10))

        # Short-Time Energy
        ste = np.mean(energy)

        # MFCCs (First 5)
        mfccs = librosa.feature.mfcc(y=y[0], sr=sr, n_mfcc=5)
        mfccs = [np.mean(mfccs[i]) for i in range(5)]

        result = {
            "Filename": os.path.basename(file_path),
            "TDOA_1_2": tdoa_1_2,
            "Zero_Crossing_Rate": zcr,
            "Spectral_Centroid": spectral_centroid,
            "Spectral_Bandwidth": spectral_bandwidth,
            "Spectral_Contrast": spectral_contrast,
            "Spectral_Rolloff": spectral_rolloff,
            "Entropy_Energy": entropy_energy,
            "Short_Time_Energy": ste,
            "MFCC_1": mfccs[0],
            "MFCC_2": mfccs[1],
            "MFCC_3": mfccs[2],
            "MFCC_4": mfccs[3],
            "MFCC_5": mfccs[4]
        }

        return result

    except Exception as e:
        print(f"Skipping {file_path} due to error: {e}")
        return None

def process_directory(input_dir, output_csv):
    data = []
    wav_files = [f for f in os.listdir(input_dir) if f.endswith('.wav')]

    for file in tqdm(wav_files, desc="Processing WAV files"):
        file_path = os.path.join(input_dir, file)
        result = extract_features(file_path)
        if result:
            data.append(result)

        # Free up memory
        gc.collect()

    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    print(f"\nFeatures saved to {output_csv}")

# Path to directory containing WAV files
INPUT_DIR = "test/AK-12"
OUTPUT_CSV =['AK-12',
 'AK-47',
 'IMI Desert Eagle',
 'M16',
 'M249',
 'M4',
 'MG-42',
 'MP5',
 'Zastava M92']
dir_list=['AK-12',
 'AK-47',
 'IMI Desert Eagle',
 'M16',
 'M249',
 'M4',
 'MG-42',
 'MP5',
 'Zastava M92']
# process_directory(INPUT_DIR, OUTPUT_CSV)
for i in range(len(dir_list)):
    process_directory('test/'+dir_list[i], OUTPUT_CSV[i]+".csv")



Processing WAV files:   0%|          | 0/98 [00:00<?, ?it/s]C:\Users\jadit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=2
  warnings.warn(
Processing WAV files: 100%|██████████| 98/98 [00:11<00:00,  8.84it/s]



Features saved to AK-12.csv


Processing WAV files:  31%|███       | 22/72 [00:02<00:05,  9.15it/s]C:\Users\jadit\AppData\Local\Temp\ipykernel_13580\585871090.py:52: RuntimeWarning: invalid value encountered in divide
  prob_energy = energy / np.sum(energy)
Processing WAV files: 100%|██████████| 72/72 [00:07<00:00,  9.11it/s]



Features saved to AK-47.csv


Processing WAV files: 100%|██████████| 100/100 [00:11<00:00,  9.08it/s]


...


Features saved to MG-42.csv


Processing WAV files: 100%|██████████| 100/100 [00:10<00:00,  9.16it/s]



Features saved to MP5.csv


Processing WAV files: 100%|██████████| 82/82 [00:08<00:00,  9.20it/s]


Features saved to Zastava M92.csv


In [9]:
os.listdir('test/')

['AK-12',
 'AK-47',
 'IMI Desert Eagle',
 'M16',
 'M249',
 'M4',
 'MG-42',
 'MP5',
 'Zastava M92']

In [12]:
import os
import pandas as pd
import glob

# Path to the directory containing CSV files
input_dir = 'ALLCSV'
output_file = 'combined_features.csv'

# Get all CSV files in the directory
csv_files = glob.glob(os.path.join(input_dir, '*.csv'))

# Read and combine all CSVs
combined_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# Save to a new CSV file
combined_df.to_csv(output_file, index=False)

print(f"Combined CSV saved to {output_file}")


Combined CSV saved to combined_features.csv
